In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

In [2]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [3]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

# SR inclusive-BDT (trained on [5,100]GeV): run on all signal mass points [5, 100] GeV

## Background

In [4]:
prefix = './coffea/skimmed/'
suffix = '_SR_BDTv5_BestVertex'

In [5]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [6]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000000
Diboson,0.000637,0.000044,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000000
Triboson,0.001933,0.000158,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000000
W+jets,0.002310,0.000173,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000000
Z+jets,0.006645,0.000398,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDTv5_Background_Efficiency'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_BDTv5_Background_Efficiency.csv


In [8]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,"38,039.83","1,882.67","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,559.22,349.76,314.03,312.99,312.95,294.00,0.12
DY,"14,495.72","2,373.75","2,373.75","2,325.57","2,325.57","2,325.57","2,325.57","2,157.97",0.00
Diboson,"4,424.20",307.91,307.91,227.24,213.36,211.27,210.33,189.35,0.58
Triboson,53.00,4.32,4.32,2.85,2.51,2.51,2.50,2.09,0.00
W+jets,"227,212.60","17,009.81","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",16.73
Z+jets,"142,220.15","8,524.55","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",2.23
Total,"432,113.00","30,662.23","30,662.23","24,835.58","23,737.17","23,695.95","23,689.22","20,736.81",19.66


In [9]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDTv5_Background_EventCount_Weighted'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDTv5_Background_EventCount_Weighted.csv


In [10]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,12311,381,381,160,159,159,158,3,0
Top,49818,4688,4688,3058,2348,2337,2336,2006,3
DY,205,14,14,12,12,12,12,10,0
Diboson,18067,1234,1234,915,860,852,848,767,2
Triboson,2257,170,170,116,106,105,104,92,0
W+jets,697657,48425,48425,37015,35472,35415,35377,29090,47
Z+jets,1281459,67427,67427,52627,50986,50934,50903,44480,18
Total,2061774,122339,122339,93903,89943,89814,89738,76448,70


In [11]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDTv5_Background_EventCount_Raw'

df_to_save = bkg_cts_raw_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDTv5_Background_EventCount_Raw.csv


In [12]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 19.66


In [13]:
pd.options.display.float_format = '{:,.2f}'.format

bkg_composition = pd.concat([pd.DataFrame(bkg_cts_df['BDT']).transpose(), pd.DataFrame(bkg_cts_raw_df['BDT']).transpose()], ignore_index=True)

bkg_composition = bkg_composition.rename(index={0: 'Weighted Counts', 1: 'Raw Counts'})
bkg_composition

,QCD,Top,DY,Diboson,Triboson,W+jets,Z+jets,Total
Weighted Counts,0.00,0.12,0.00,0.58,0.00,16.73,2.23,19.66
Raw Counts,0.00,3.00,0.00,2.00,0.00,47.00,18.00,70.00


## Signal

In [14]:
# Signal
sig_histo = util.load(f'{prefix}/signal{suffix}.coffea')[0]

In [15]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,99.0,18.0,100.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-100
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,94.5,9.0,100.0,90.0,99.0,0.1,sig_2018_Mchi-94p5_dMchi-9p0_ctau-100
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,99.0,18.0,10.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-10
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,99.0,18.0,1.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-1
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,88.0,16.0,100.0,80.0,96.0,0.2,sig_2018_Mchi-88p0_dMchi-16p0_ctau-100
...,...,...,...,...,...,...,...
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.5,1.0,10.0,10.0,11.0,0.1,sig_2018_Mchi-10p5_dMchi-1p0_ctau-10
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.5,1.0,1.0,10.0,11.0,0.1,sig_2018_Mchi-10p5_dMchi-1p0_ctau-1
sig_2018_Mchi-105p0_dMchi-10p0_ctau-100,105.0,10.0,100.0,100.0,110.0,0.1,sig_2018_Mchi-105p0_dMchi-10p0_ctau-100
sig_2018_Mchi-105p0_dMchi-10p0_ctau-10,105.0,10.0,10.0,100.0,110.0,0.1,sig_2018_Mchi-105p0_dMchi-10p0_ctau-10


In [16]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'computing BDT score',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT']

### Efficiency

In [17]:
pd.options.display.float_format = '{:,.6f}'.format

In [18]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [19]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [20]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,0.073536,0.022607,0.022607,0.019257,0.018700,0.018696,0.018696,0.018507,0.010649
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,0.069469,0.015265,0.015265,0.013049,0.012632,0.012628,0.012628,0.012513,0.005930
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,0.060691,0.012979,0.012979,0.011187,0.010795,0.010792,0.010792,0.010711,0.004980
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,0.055333,0.016089,0.016089,0.013636,0.013223,0.013223,0.013223,0.013076,0.007982
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,0.053240,0.010559,0.010559,0.008952,0.008713,0.008713,0.008713,0.008624,0.004008
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,0.044809,0.008682,0.008682,0.007384,0.007083,0.007083,0.007083,0.007008,0.003143
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014005,0.001968,0.001968,0.001649,0.001547,0.001543,0.001543,0.001533,0.000425
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,0.013839,0.001680,0.001680,0.001507,0.001444,0.001444,0.001444,0.001431,0.000125
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.039025,0.010893,0.010893,0.009249,0.008983,0.008983,0.008983,0.008894,0.005916
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,0.037325,0.006925,0.006925,0.005968,0.005759,0.005757,0.005757,0.005730,0.002442


In [21]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDTv5_Signal_ctau-1_samples_Efficiency.csv


In [22]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,0.064370,0.015743,0.015743,0.013400,0.012993,0.012986,0.012986,0.012779,0.007643
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,0.055466,0.013320,0.013320,0.011199,0.010929,0.010922,0.010922,0.010763,0.006819
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,0.060137,0.010865,0.010865,0.009205,0.008914,0.008909,0.008909,0.008806,0.004464
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,0.053515,0.009353,0.009353,0.008067,0.007793,0.007790,0.007790,0.007726,0.003679
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,0.048804,0.011110,0.011110,0.009386,0.009098,0.009094,0.009094,0.008954,0.005343
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,0.046143,0.007822,0.007822,0.006711,0.006462,0.006459,0.006459,0.006378,0.003110
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,0.039471,0.006180,0.006180,0.005317,0.005114,0.005107,0.005107,0.005050,0.002148
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010583,0.001135,0.001135,0.000939,0.000886,0.000886,0.000886,0.000845,0.000150
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,0.010073,0.000905,0.000905,0.000755,0.000711,0.000711,0.000711,0.000689,0.000054
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.033117,0.007081,0.007081,0.006045,0.005778,0.005778,0.005778,0.005709,0.003536


In [23]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDTv5_Signal_ctau-10_samples_Efficiency.csv


In [24]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.042940,0.005995,0.005995,0.005063,0.004896,0.004892,0.004892,0.004715,0.001918
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,0.041015,0.004814,0.004814,0.004086,0.003980,0.003979,0.003979,0.003889,0.001245
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.038229,0.005134,0.005134,0.004414,0.004249,0.004245,0.004245,0.004080,0.001673
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,0.036446,0.004232,0.004232,0.003565,0.003451,0.003444,0.003444,0.003364,0.001025
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.032224,0.004127,0.004127,0.003473,0.003328,0.003325,0.003325,0.003209,0.001274
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,0.032000,0.003520,0.003520,0.002984,0.002896,0.002893,0.002893,0.002799,0.000731
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,0.026468,0.002693,0.002693,0.002280,0.002172,0.002172,0.002172,0.002124,0.000585
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007029,0.000627,0.000627,0.000548,0.000518,0.000518,0.000518,0.000490,0.000031
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.006932,0.000571,0.000571,0.000482,0.000453,0.000453,0.000453,0.000435,0.000008
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.022342,0.002711,0.002711,0.002289,0.002183,0.002183,0.002183,0.002106,0.000764


In [25]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDTv5_Signal_ctau-100_samples_Efficiency.csv


### Weighted Event count

In [26]:
pd.options.display.float_format = '{:,.2f}'.format

In [27]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [28]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [29]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,0.10,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.01
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,530.03,116.47,116.47,99.56,96.38,96.35,96.35,95.47,45.24
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,634.96,135.79,135.79,117.04,112.94,112.90,112.90,112.06,52.10
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,0.18,0.05,0.05,0.05,0.04,0.04,0.04,0.04,0.03
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,799.43,158.55,158.55,134.43,130.83,130.83,130.83,129.49,60.18
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,"1,098.29",212.80,212.80,180.99,173.61,173.61,173.61,171.77,77.03
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,17.44,2.45,2.45,2.05,1.93,1.92,1.92,1.91,0.53
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"3,492.09",424.04,424.04,380.17,364.43,364.43,364.43,361.05,31.50
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.62,0.17,0.17,0.15,0.14,0.14,0.14,0.14,0.09
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"2,331.91",432.67,432.67,372.86,359.82,359.66,359.66,357.95,152.57


In [30]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDTv5_Signal_ctau-1_samples_EventCount_Weighted.csv


In [31]:
# without m1 = 30 GeV
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,0.10,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.01
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,530.03,116.47,116.47,99.56,96.38,96.35,96.35,95.47,45.24
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,634.96,135.79,135.79,117.04,112.94,112.90,112.90,112.06,52.10
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,0.18,0.05,0.05,0.05,0.04,0.04,0.04,0.04,0.03
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,799.43,158.55,158.55,134.43,130.83,130.83,130.83,129.49,60.18
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,"1,098.29",212.80,212.80,180.99,173.61,173.61,173.61,171.77,77.03
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,17.44,2.45,2.45,2.05,1.93,1.92,1.92,1.91,0.53
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"3,492.09",424.04,424.04,380.17,364.43,364.43,364.43,361.05,31.50
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.62,0.17,0.17,0.15,0.14,0.14,0.14,0.14,0.09
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"2,331.91",432.67,432.67,372.86,359.82,359.66,359.66,357.95,152.57


In [32]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,145.37,26.26,26.26,22.25,21.55,21.54,21.54,21.29,10.79
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,177.46,31.02,31.02,26.75,25.84,25.83,25.83,25.62,12.20
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,220.50,37.38,37.38,32.07,30.88,30.86,30.86,30.48,14.86
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,307.15,48.09,48.09,41.38,39.79,39.74,39.74,39.30,16.71
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,1.32,0.14,0.14,0.12,0.11,0.11,0.11,0.11,0.02
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,714.51,64.21,64.21,53.53,50.46,50.46,50.46,48.85,3.80
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.05,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [33]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDTv5_Signal_ctau-10_samples_EventCount_Weighted.csv


In [34]:
# without m1 = 30 GeV
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,145.37,26.26,26.26,22.25,21.55,21.54,21.54,21.29,10.79
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,177.46,31.02,31.02,26.75,25.84,25.83,25.83,25.62,12.20
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,220.50,37.38,37.38,32.07,30.88,30.86,30.86,30.48,14.86
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,307.15,48.09,48.09,41.38,39.79,39.74,39.74,39.30,16.71
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,1.32,0.14,0.14,0.12,0.11,0.11,0.11,0.11,0.02
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,714.51,64.21,64.21,53.53,50.46,50.46,50.46,48.85,3.80
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.05,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [35]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,68.30,8.02,8.02,6.80,6.63,6.63,6.63,6.48,2.07
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,76.02,8.83,8.83,7.44,7.20,7.18,7.18,7.02,2.14
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,84.00,9.24,9.24,7.83,7.60,7.59,7.59,7.35,1.92
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,88.95,9.05,9.05,7.66,7.30,7.30,7.30,7.14,1.97
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.09,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,150.47,12.39,12.39,10.46,9.84,9.82,9.82,9.44,0.18
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [36]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDTv5_Signal_ctau-100_samples_EventCount_Weighted.csv


In [37]:
# without m1 = 30 GeV
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,68.30,8.02,8.02,6.80,6.63,6.63,6.63,6.48,2.07
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,76.02,8.83,8.83,7.44,7.20,7.18,7.18,7.02,2.14
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,84.00,9.24,9.24,7.83,7.60,7.59,7.59,7.35,1.92
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,88.95,9.05,9.05,7.66,7.30,7.30,7.30,7.14,1.97
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.09,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,150.47,12.39,12.39,10.46,9.84,9.82,9.82,9.44,0.18
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Raw count

In [38]:
pd.options.display.float_format = '{:,.0f}'.format

In [39]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [40]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [41]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,36268,11150,11150,9498,9223,9221,9221,9128,5252
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,52076,11443,11443,9782,9469,9466,9466,9380,4445
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,18119,3875,3875,3340,3223,3222,3222,3198,1487
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,15806,4596,4596,3895,3777,3777,3777,3735,2280
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,16206,3214,3214,2725,2652,2652,2652,2625,1220
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,13703,2655,2655,2258,2166,2166,2166,2143,961
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4120,579,579,485,455,454,454,451,125
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3104,377,377,338,324,324,324,321,28
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,878,245,245,208,202,202,202,200,133
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,15009,2785,2785,2400,2316,2315,2315,2304,982


In [42]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDTv5_Signal_ctau-1_samples_EventCount_Raw.csv


In [43]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,34204,8365,8365,7120,6904,6900,6900,6790,4061
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,8028,1928,1928,1621,1582,1581,1581,1558,987
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,50576,9138,9138,7742,7497,7493,7493,7406,3754
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,15814,2764,2764,2384,2303,2302,2302,2283,1087
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,13868,3157,3157,2667,2585,2584,2584,2544,1518
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,13709,2324,2324,1994,1920,1919,1919,1895,924
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,12404,1942,1942,1671,1607,1605,1605,1587,675
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,2817,302,302,250,236,236,236,225,40
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,4886,439,439,366,345,345,345,334,26
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,3840,821,821,701,670,670,670,662,410


In [44]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDTv5_Signal_ctau-10_samples_EventCount_Raw.csv


In [45]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,23278,3250,3250,2745,2654,2652,2652,2556,1040
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,30269,3553,3553,3015,2937,2936,2936,2870,919
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,10627,1427,1427,1227,1181,1180,1180,1134,465
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,10877,1263,1263,1064,1030,1028,1028,1004,306
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,9158,1173,1173,987,946,945,945,912,362
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,9810,1079,1079,915,888,887,887,858,224
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,8276,842,842,713,679,679,679,664,183
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2064,184,184,161,152,152,152,144,9
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,10156,836,836,706,664,663,663,637,12
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,6345,770,770,650,620,620,620,598,217


In [46]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDTv5_Signal_ctau-{int(ct)}_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDTv5_Signal_ctau-100_samples_EventCount_Raw.csv


## S/sqrt(B)

In [47]:
nBkg

19.657784520445524

In [48]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [49]:
pd.options.display.float_format = '{:,.4f}'.format

In [50]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [51]:
# Missing rows
#df[ (df['m1'] == 60) & (df['delta'] == 0.2) ]

df.loc[len(df.index)] = [0, 60, 0.2, 1]
df.loc[len(df.index)] = [0, 60, 0.2, 10]
df.loc[len(df.index)] = [0, 60, 0.2, 100]
df.loc[len(df.index)] = [0, 80, 0.2, 1] 

df = df.sort_values(by=['m1']) # sort by m1

In [52]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,3.8027,5.0000,0.1000,10.0000,19.6578
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,31.5013,5.0000,0.1000,1.0000,19.6578
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.1778,5.0000,0.1000,100.0000,19.6578
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.5292,5.0000,0.2000,1.0000,19.6578
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.0187,5.0000,0.2000,10.0000,19.6578
...,...,...,...,...,...
sig_2018_Mchi-110p0_dMchi-20p0_ctau-1,0.0111,100.0000,0.2000,1.0000,19.6578
sig_2018_Mchi-105p0_dMchi-10p0_ctau-100,2.0151,100.0000,0.1000,100.0000,19.6578
sig_2018_Mchi-105p0_dMchi-10p0_ctau-10,9.3727,100.0000,0.1000,10.0000,19.6578
sig_2018_Mchi-105p0_dMchi-10p0_ctau-1,39.6619,100.0000,0.1000,1.0000,19.6578


In [53]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,3.8027,19.6578,0.8577
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,31.5013,19.6578,7.1050
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,0.1778,19.6578,0.0401
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,0.5292,19.6578,0.1194
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,0.0187,19.6578,0.0042
...,...,...,...,...,...,...
sig_2018_Mchi-110p0_dMchi-20p0_ctau-1,100.0000,0.2000,1.0000,0.0111,19.6578,0.0025
sig_2018_Mchi-105p0_dMchi-10p0_ctau-100,100.0000,0.1000,100.0000,2.0151,19.6578,0.4545
sig_2018_Mchi-105p0_dMchi-10p0_ctau-10,100.0000,0.1000,10.0000,9.3727,19.6578,2.1140
sig_2018_Mchi-105p0_dMchi-10p0_ctau-1,100.0000,0.1000,1.0000,39.6619,19.6578,8.9455


In [54]:
outdir = './csv/SR_Yields/'
outname = f'SR_BDTv5_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_BDTv5_S_over_B.csv


### ctau = 1mm signal points

In [55]:
ct = 1.0

In [56]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.1
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0,0.1,1.0,31.5013,19.6578,7.1050
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0,0.1,1.0,62.6205,19.6578,14.1237
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0,0.1,1.0,80.0488,19.6578,18.0546
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0,0.1,1.0,110.4298,19.6578,24.9069
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0,0.1,1.0,943.7997,19.6578,212.8691
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,50.0,0.1,1.0,152.5696,19.6578,34.4113
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,60.0,0.1,1.0,77.0253,19.6578,17.3727
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,70.0,0.1,1.0,60.1836,19.6578,13.5741
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,80.0,0.1,1.0,52.1019,19.6578,11.7513
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,90.0,0.1,1.0,45.2422,19.6578,10.2042


In [57]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.2
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0,0.2,1.0,0.5292,19.6578,0.1194
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0,0.2,1.0,0.5576,19.6578,0.1258
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0,0.2,1.0,0.7815,19.6578,0.1763
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0,0.2,1.0,1.4754,19.6578,0.3328
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0,0.2,1.0,72.8313,19.6578,16.4267
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,50.0,0.2,1.0,0.0933,19.6578,0.0210
62,60.0,0.2,1.0,0.0000,19.6578,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,70.0,0.2,1.0,0.0264,19.6578,0.0060
65,80.0,0.2,1.0,0.0000,19.6578,0.0000
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,90.0,0.2,1.0,0.0149,19.6578,0.0034


### ctau = 10mm signal points

In [58]:
ct = 10.0

In [59]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 10mm, delta = 0.1
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0,0.1,10.0,3.8027,19.6578,0.8577
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0,0.1,10.0,9.5345,19.6578,2.1504
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0,0.1,10.0,14.5929,19.6578,3.2913
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0,0.1,10.0,24.9082,19.6578,5.6179
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0,0.1,10.0,73.1155,19.6578,16.4908
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,50.0,0.1,10.0,35.9580,19.6578,8.1101
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,60.0,0.1,10.0,16.7146,19.6578,3.7699
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,70.0,0.1,10.0,14.8610,19.6578,3.3518
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,80.0,0.1,10.0,12.1985,19.6578,2.7513
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,90.0,0.1,10.0,10.7896,19.6578,2.4335


In [60]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 10mm, delta = 0.2
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0,0.2,10.0,0.0187,19.6578,0.0042
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0,0.2,10.0,0.0291,19.6578,0.0066
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0,0.2,10.0,0.0403,19.6578,0.0091
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0,0.2,10.0,0.0912,19.6578,0.0206
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0,0.2,10.0,4.5489,19.6578,1.0260
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,50.0,0.2,10.0,0.0056,19.6578,0.0013
63,60.0,0.2,10.0,0.0000,19.6578,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,70.0,0.2,10.0,0.0018,19.6578,0.0004
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,80.0,0.2,10.0,0.0014,19.6578,0.0003
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,90.0,0.2,10.0,0.0011,19.6578,0.0002


### ctau = 100mm signal points

In [61]:
ct = 100.0

In [62]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 100mm, delta = 0.1
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0,0.1,100.0,0.1778,19.6578,0.0401
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0,0.1,100.0,0.5013,19.6578,0.1131
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0,0.1,100.0,0.7271,19.6578,0.1640
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0,0.1,100.0,1.3475,19.6578,0.3039
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0,0.1,100.0,1.6659,19.6578,0.3757
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,50.0,0.1,100.0,2.6941,19.6578,0.6076
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,60.0,0.1,100.0,1.9675,19.6578,0.4438
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,70.0,0.1,100.0,1.9183,19.6578,0.4327
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,80.0,0.1,100.0,2.1388,19.6578,0.4824
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,90.0,0.1,100.0,2.0737,19.6578,0.4677


In [63]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 100mm, delta = 0.2
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0,0.2,100.0,0.0004,19.6578,0.0001
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0,0.2,100.0,0.0004,19.6578,0.0001
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0,0.2,100.0,0.0008,19.6578,0.0002
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0,0.2,100.0,0.0018,19.6578,0.0004
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0,0.2,100.0,0.1003,19.6578,0.0226
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,50.0,0.2,100.0,0.0001,19.6578,0.0000
64,60.0,0.2,100.0,0.0000,19.6578,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,70.0,0.2,100.0,0.0000,19.6578,0.0000
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,80.0,0.2,100.0,0.0000,19.6578,0.0000
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,90.0,0.2,100.0,0.0000,19.6578,0.0000
